In [95]:
import requests
import pandas as pd
from pandas import DataFrame 
%matplotlib inline
import matplotlib.pyplot as plt
import zipfile
import numpy as np
import io
import time
from numpy import nan
from pprint import pprint 
print("Import Successful")
#!wget https://www150.statcan.gc.ca/n1/tbl/csv/10100084-eng.zip
#!unzip 10100084-eng.zip

Import Successful


# Downloading the data

In [96]:



df = pd.read_csv("TorontoListings.csv")


# Data cleaning

In [97]:
df.describe()

,id,host_id,neighbourhood_group,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.874600e+04,1.874600e+04,0.0,18746.000000,18746.000000,18746.000000,18746.000000,18746.000000,14788.000000,18746.000000,18746.000000
mean,2.673976e+07,1.087340e+08,NaN,43.680717,-79.397420,135.468473,10.786568,26.635602,1.214266,4.715246,125.690761
std,1.298332e+07,1.055992e+08,NaN,0.048263,0.064453,263.761517,36.913904,52.418576,1.597121,10.354866,136.844548
min,1.419000e+03,1.565000e+03,NaN,43.586710,-79.634800,11.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,1.634553e+07,2.049040e+07,NaN,43.645170,-79.426310,61.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,2.775978e+07,6.787450e+07,NaN,43.662815,-79.397350,99.000000,2.000000,6.000000,0.600000,1.000000,83.000000
75%,3.853432e+07,1.823192e+08,NaN,43.700153,-79.376860,150.000000,7.000000,27.000000,1.580000,3.000000,252.000000
max,4.578971e+07,3.711952e+08,NaN,43.836900,-79.127810,13137.000000,1125.000000,828.000000,15.440000,88.000000,365.000000


From above, We can see there is a column which is empty, Hence, We drop empty, NAN data from the set. 

In [98]:

df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group               float64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

Since id and host_name, both are giving the same information, We keep one of them and We will do the same thing for name and coordination(lattitude+longitude), We will concatenate them into coulumn location

In [99]:
df.drop(['neighbourhood_group', 'name', 'host_name'], 1, inplace = True)
df.head()

,id,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1419,1565,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0
1,8077,22795,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365
2,12604,48239,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0
3,23691,93825,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240
4,26654,113345,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295


In [100]:
df['location'] = [', '.join(str(x) for x in y) for y in map(tuple, df[['latitude', 'longitude']].values)]
df.head(20)

,id,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,location
0,1419,1565,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0,"43.64617, -79.42451"
1,8077,22795,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365,"43.64105, -79.37628000000001"
2,12604,48239,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0,"43.66724, -79.41598"
3,23691,93825,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240,"43.696020000000004, -79.45468000000001"
4,26654,113345,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295,"43.6453, -79.3894"
5,27423,118124,Greenwood-Coxwell,43.66890,-79.32592,Entire home/apt,54,120,26,2011-08-30,0.21,1,0,"43.6689, -79.32592"
6,28160,86838,Mount Pleasant West,43.70376,-79.39077,Entire home/apt,50,60,7,2018-10-17,0.11,1,364,"43.703759999999996, -79.39076999999999"
7,30931,22795,Waterfront Communities-The Island,43.64151,-79.37643,Entire home/apt,131,180,1,2010-08-11,0.01,2,365,"43.64151, -79.37643"
8,40456,174063,South Parkdale,43.63532,-79.44049,Entire home/apt,100,30,110,2020-03-25,0.88,5,359,"43.63532, -79.44049"
9,41887,183071,Oakridge,43.69466,-79.28667,Entire home/apt,75,28,82,2019-09-02,1.74,2,342,"43.69466, -79.28667"


# Taking care of missing data

In [101]:
df = df[['id', 'host_id', 'neighbourhood', 'latitude', 'longitude', 'room_type','minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'location', 'price']]

In [102]:
# replace '0' values with 'nan'
df[['number_of_reviews', 'availability_365']] = df[['number_of_reviews', 'availability_365']].replace(0, nan)
# count the number of nan values in each column
print(df.isnull().sum())

id                                   0
host_id                              0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                 3958
last_review                       3958
reviews_per_month                 3958
calculated_host_listings_count       0
availability_365                  6641
location                             0
price                                0
dtype: int64


In [103]:
#conversion of the 'REF_DATE' from a string to a proper datetime object.
df['last_review'] = pd.to_datetime(df['last_review']) 
# fill missing values with mean column values
df.fillna(df.mean(), inplace=True)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  after removing the cwd from sys.path.


# Encoding categorical Data

In [104]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
X=df.iloc[: , 5:12].values
y=df.iloc[:, 13].values
one_hot = pd.get_dummies(df['room_type'])


In [105]:
one_hot_df = pd.concat([df, one_hot], axis=1)


In [106]:
one_hot_df = one_hot_df[['id', 'host_id', 'neighbourhood','minimum_nights', 'number_of_reviews',
       'calculated_host_listings_count', 'availability_365', 'reviews_per_month','latitude', 'longitude','Shared room', 'Entire home/apt',
       'Hotel room', 'Private room', 'price']]


In [107]:
#Encoding neighbourhood
one_hot_neighbourhood = pd.get_dummies(df['neighbourhood'])


In [108]:
one_hot_encoded_df = pd.concat([one_hot_df, one_hot_neighbourhood], axis=1)

one_hot_encoded_df["reviews_per_month"].mean()

filtered_df = one_hot_encoded_df[one_hot_encoded_df['reviews_per_month'] >= 1.2] 

In [109]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])
filtered_df = movecol(filtered_df, 
             cols_to_move=['Yorkdale-Glen Park','price'], 
             ref_col='York University Heights',
             place='After')


# Definig dataset for the models

In the next lines, I am going to categorize the prices as low, affordable and expensive. Then then the model willl be trained sperately on each class.10 columns were selected to start with building our model and see how it works.

In [110]:
def priceRanker(val):
    if  val< 100:
        return 1
    elif 200 < val:
         return 2
    else :
         return 3

filtered_df["categoryPrice"] = df["price"].apply(priceRanker)
   

In [111]:
filtered_df

,id,host_id,neighbourhood,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,reviews_per_month,latitude,longitude,Shared room,Entire home/apt,Hotel room,Private room,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St.James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,...,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pellam Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park,price,categoryPrice
1,8077,22795,Waterfront Communities-The Island,180,169.000000,2,365.000000,1.240000,43.64105,-79.37628,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,98,1
2,12604,48239,Annex,1,33.764606,1,194.646758,1.214266,43.66724,-79.41598,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,66,1
3,23691,93825,Briar Hill-Belgravia,1,217.000000,2,240.000000,1.720000,43.69602,-79.45468,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,1
9,41887,183071,Oakridge,28,82.000000,2,342.000000,1.740000,43.69466,-79.28667,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,75,1
10,42892,187320,Waterfront Communities-The Island,29,33.764606,13,265.000000,1.214266,43.64451,-79.38185,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,110,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18741,45785334,1023135,Yonge-Eglinton,90,33.764606,1,344.000000,1.214266,43.70703,-79.40052,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,65,1
18742,45786222,371188170,Newtonbrook East,90,33.764606,1,179.000000,1.214266,43.79075,-79.39699,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,158,3
18743,45786961,371195218,Bayview Village,1,33.764606,1,194.646758,1.214266,43.76668,-79.37158,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,1
18744,45789233,304805850,Corso Italia-Davenport,1,33.764606,2,52.000000,1.214266,43.67330,-79.44542,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,1


# SMOTE for imbalanced classification

if we go through the dataset after categorizing and applying the PriceRanker, it can be seen that there is an imblanced available data in mid range prices. To solve the problem we are going to deploy SMOTE.

In [112]:
!pip install  imbalanced-learn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
#define dataset
X_df = filtered_df.iloc[: , 7:-2]
X = X_df.values
y =filtered_df.iloc[:, -1].values
#Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.9, random_state = 0)
#SMOTE
sm = SMOTE()
X, y = sm.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


# Decision Tree Cross Validation

For cross validation purposes, I am going to train 90% of our dataset.

---



now I need to define a new classifier to fit on our cross validation dataset. which I call it clf_cv

# Hyperparameter tuning for DT

In [113]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

acc_score_DT = []
best_score_DT = (0, -1)
best_clf_DT = None

for i in range(1,21):
    clf_DT = DecisionTreeClassifier(max_depth=i,  random_state=0)
    clf_DT.fit(X_train, y_train)
    y_pred_DT=clf_DT.predict(X_test)
    
    curr_acc_DT = accuracy_score(y_test, y_pred_DT)
    acc_score_DT.append((i, curr_acc_DT))
    
    if curr_acc_DT > best_score_DT[0]:
        best_score_DT = (curr_acc_DT, i)
        best_clf_DT = clf_DT

print(clf_DT)
acc_score_DT

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=20, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')


[(1, 0.610241273100616),
 (2, 0.6116529774127311),
 (3, 0.627053388090349),
 (4, 0.6175564681724846),
 (5, 0.608829568788501),
 (6, 0.6049794661190965),
 (7, 0.6033110882956879),
 (8, 0.5903490759753593),
 (9, 0.5922741273100616),
 (10, 0.5880390143737166),
 (11, 0.5728952772073922),
 (12, 0.5691735112936345),
 (13, 0.5619866529774127),
 (14, 0.5613449691991786),
 (15, 0.5582648870636551),
 (16, 0.5532597535934292),
 (17, 0.555441478439425),
 (18, 0.5531314168377823),
 (19, 0.5522330595482546),
 (20, 0.5572381930184805)]

## Matrix of confusion for Decision Tree cross validation

In [114]:
from sklearn.metrics import confusion_matrix
clf_DT_final = DecisionTreeClassifier(max_depth=4,random_state=0)
clf_DT_final.fit(X_train, y_train)
y_pred_DT=clf_DT_final.predict(X_test)
cm_DT = confusion_matrix(y_test, y_pred_DT)
print(cm_DT)
accuracy_score(y_test, y_pred_DT)

[[2884    0  939]
 [ 310    0  642]
 [1089    0 1928]]


0.6175564681724846

# Random Forest Classification 


for random forest model we have to define the number of features that we are including in our model that for this case it is 147 ( total number of coulumns _ 9). 

In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Convert dataset from regression to classification
X, y = make_classification(n_samples=1000, n_features=147,
                            n_informative=3, n_redundant=0,
                            random_state=0, shuffle=False)


# Hyperparameter Tuning for RF model

#For tuning I am going to train and fit the model with a different values for max_depth and n_estimator. I chose a range of 3 to 20 for max_depth and 20 to 100 for n_estimator.

In [116]:
acc_score_RF = []
best_score_RF = (0, -1)


# Create nested for loop
for i in range(6,20):
  for j in range(22,30):
    clf_RF = RandomForestClassifier(max_depth=i,n_estimators=j,random_state=0)
    clf_RF.fit(X_train, y_train)
    y_pred_RF=clf_RF.predict(X_test)
    
    curr_acc_RF = accuracy_score(y_test, y_pred_RF)
    acc_score_RF.append((i,j, curr_acc_RF))
    
    if curr_acc_RF > best_score_RF[0]:
        best_score_RF = (curr_acc_RF, i, j)
        best_clf_DT = clf_RF
        
print(best_clf_DT)



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=25,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


**bold text**# Now We need to creat a matrix of confusion for RF cross validation 

In [117]:
clf_RF = RandomForestClassifier(max_depth=15,n_estimators=27,random_state=0)
clf_RF.fit(X_train, y_train)
y_pred_RF=clf_RF.predict(X_test)
cm_RF= confusion_matrix(y_test, y_pred_RF)
print(cm_RF)
accuracy_score(y_test, y_pred_RF)

[[2795   35  993]
 [ 226   40  686]
 [ 867   51 2099]]


0.6332135523613963

# KNN Classifier

# Hyperparameter tuning for KNN

In [118]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)


##The only difference in terms of model tuning for KNN is we are changing the number of nearest numbers for a range from 4 to 20.

In [119]:
acc_score_knn = []
best_score_knn = (0, -1)

for i in range(14,35):
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train, y_train)
    y_pred_knn=neigh.predict(X_test) 
  
  
    
    curr_acc_knn = accuracy_score(y_test, y_pred_knn)
    acc_score_knn.append((i, curr_acc_knn))
    
    if curr_acc_knn > best_score_knn[0]:
        best_score_knn = (curr_acc_knn, i)
        best_clf_DT = neigh
print(best_clf_DT)
acc_score_knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=30, p=2,
                     weights='uniform')


[(14, 0.5952258726899384),
 (15, 0.5950975359342916),
 (16, 0.5934291581108829),
 (17, 0.5938141683778234),
 (18, 0.5993326488706365),
 (19, 0.598305954825462),
 (20, 0.6008726899383984),
 (21, 0.6036960985626283),
 (22, 0.6001026694045175),
 (23, 0.6022843942505134),
 (24, 0.6020277207392197),
 (25, 0.6025410677618069),
 (26, 0.6052361396303901),
 (27, 0.6058778234086243),
 (28, 0.603952772073922),
 (29, 0.6062628336755647),
 (30, 0.6063911704312115),
 (31, 0.6062628336755647),
 (32, 0.6038244353182751),
 (33, 0.6029260780287474),
 (34, 0.6058778234086243)]

# KNN confusion matrix

In [120]:
neigh = KNeighborsClassifier(n_neighbors=14)
neigh.fit(X_train, y_train)
y_pred_knn=neigh.predict(X_test) 
cm_knn= confusion_matrix(y_test, y_pred_knn)
print(cm_knn)
accuracy_score(y_test, y_pred_knn)

[[2592   50 1181]
 [ 253   38  661]
 [ 937   72 2008]]


0.5952258726899384